In [1]:
import numpy as np
import math
import pandas as pd
import keras
from keras.preprocessing import image
from PIL import Image
import os
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

E:\develop\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
flowers_path = os.listdir('data/flowers-recognition/flowers/')
flowers_path

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [15]:
paths = {}
for path in flowers_path:
    paths[path] = []
for key, images in paths.items():
    for filename in os.listdir('data/flowers-recognition/flowers/'+key):
        if '.py' not in filename:
            paths[key].append('data/flowers-recognition/flowers/'+key+'/'+filename)
    print(len(images),key,'images')

984 tulip images
734 sunflower images
784 rose images
769 daisy images
1052 dandelion images


In [17]:
img_size=28
X = []
Y = []
mapping = {}
i = 0
for path in flowers_path:
    mapping[path] = i
    i += 1
for label,image_paths in paths.items():
    for path in image_paths:
        image = Image.open(path)
        image = image.resize((img_size, img_size))
        X.append(np.array(image))
        Y.append(mapping[label])

In [18]:
X = np.array(X).astype('float32')
X = X / 255

In [20]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, shuffle=True, random_state=0)

In [22]:
x_train = np.reshape(x_train, (-1, 3, 28, 28))
x_test = np.reshape(x_test, (-1, 3, 28, 28))
x_train = torch.Tensor(x_train).float()
y_train = torch.Tensor(y_train).long()
tensorset = TensorDataset(x_train, y_train)
trainloader = DataLoader(tensorset, batch_size=128, shuffle=True, num_workers=2)
x_test = torch.Tensor(x_test).float()
y_test = torch.Tensor(y_test).long()
tensorset = TensorDataset(x_test, y_test)
testloader = DataLoader(tensorset, batch_size=128, shuffle=True, num_workers=2)
dataloaders = {'train': trainloader, 'val': testloader}

### cuda

In [4]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
device = torch.device(device)

### Model

In [55]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, drop_rate=0.0):
        super(BasicBlock, self).__init__()
        
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                              stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size,
                              stride=1, padding=1, bias=False)
        self.dropRate = drop_rate
        self.equalInOut = (in_channels == out_channels)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None
    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.dropRate > 0:
            out = F.dropout(out,p=self.dropRate, training=self.training)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)

class NetworkBlock(nn.Module):
    def __init__(self,block, n_layers, in_channels, out_channels, kernel_size=3, stride=1, drop_rate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, n_layers, in_channels, out_channels, kernel_size, stride=stride, drop_rate=drop_rate)
        
    def _make_layer(self, block, n_layers, in_channels, out_channels, kernel_size, stride, drop_rate):
        layers = []
        for i in range(n_layers):
            layers.append(block(i == 0 and in_channels or out_channels, out_channels, kernel_size, i == 0 and stride or 1, drop_rate))
        return nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layer(x)
    
class WideResNet(nn.Module):
    def __init__(self, depth, num_classes, in_channels=1, wide_factor=1, drop_rate=0.0):
        super(WideResNet, self).__init__()
        channels = [16, 16 * wide_factor, 32 * wide_factor, 64 * wide_factor]
        assert (depth - 4) % 6 == 0,'depth should 6n+4'
        n = int((depth - 4) / 6)
        block = BasicBlock
        
        self.conv1 = nn.Conv2d(in_channels, channels[0], kernel_size=3, padding=1, bias=False)
        self.block1 = NetworkBlock(block, n, channels[0], channels[1], kernel_size=3, stride=1, drop_rate=drop_rate)
        self.block2 = NetworkBlock(block, n, channels[1], channels[2], kernel_size=3, stride=2, drop_rate=drop_rate)
        self.block3 = NetworkBlock(block, n, channels[2], channels[3], kernel_size=3, stride=2, drop_rate=drop_rate)
        self.bn = nn.BatchNorm2d(channels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(channels[3], num_classes)
        self.channels = channels[3]
        
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
                
    def forward(self, x):
        x = self.conv1(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.relu(self.bn(x))
        x = F.avg_pool2d(x, 7)
        x = x.view(-1, self.channels)
        x = self.fc(x)
        return x

def wrn(depth, num_classes, in_channels=1, wide_factor=1, drop_rate=0.0):
    model = WideResNet(depth, num_classes, in_channels=in_channels)
    return model

In [59]:
num_epochs = 28
dataset_sizes = {'train':len(x_train), 'val': len(x_test)}
model = wrn(num_epochs, 5, in_channels=3)
model.to(device)

WideResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (block1): NetworkBlock(
    (layer): Sequential(
      (0): BasicBlock(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): BasicBlock(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
      

In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [ ]:
best_acc = 0.0
num_epochs = 128
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 30)
    for phase in ['train', 'val']:
        running_loss = 0.0
        running_corrects = 0
        
        for i, data in enumerate(dataloaders[phase], 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            #forward
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            #backward
            if phase == 'train':
                loss.backward()
                optimizer.step()
            running_loss += loss.item()
            
            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
print('Finished Training')

Epoch 0/127
------------------------------
train Loss: 1.6003 Acc: 0.2686
val Loss: 1.5760 Acc: 0.3326
Epoch 1/127
------------------------------
train Loss: 1.5177 Acc: 0.3344
val Loss: 1.5028 Acc: 0.3672
Epoch 2/127
------------------------------
train Loss: 1.4614 Acc: 0.3602
val Loss: 1.4385 Acc: 0.3811
Epoch 3/127
------------------------------
